In [ ]:
### note this notebook is based off of one I found on one of the DAQ machines at CERN

# define some functions

from oei import *
from tqdm.notebook import trange, tqdm
import matplotlib.pyplot as plt
import numpy as np
import warnings
import time

class readwave:
    def __init__(self, ep=100, afe=0, ch=0,plot=False):
        thing = OEI(f"10.73.137.{ep}")
        thing.write(0x2000,[1234])
        wf=[]
        for i in range (20):
            doutrec = thing.read(0x40000000+(0x100000 * afe)+(0x10000 * ch)+i*50,50)[2:]
            for word in doutrec:
                wf.append(word)
        #print(len(wf))
        x = np.linspace(0.0, len(wf) * 16e-9, num=len(wf))
        if plot:
            plt.figure()
            plt.plot(x,wf,linewidth=0.6,label=f' \t rms={np.round(np.mean(wf),2)}')
            plt.show()
        self.wf=np.array(wf)
        self.afe=afe
        self.ch=ch
        self.ep=ep
        thing.close()

        
class fft:
    def __init__(self, sig, dt=16e-9, plot=False):
        np.seterr(divide = 'ignore') 
        if dt is None:
            dt = 1
            t = np.arange(0, sig.shape[-1])
            xLabel = "samples"
        else:
            t = np.arange(0, sig.shape[-1]) * dt
            xLabel = "freq [Hz]"

        if sig.shape[0] % 2 != 0:
            warnings.warn("signal preferred to be even in size, autoFixing it...")
            t = t[:-1]
            sig = sig[:-1]

        sigFFT = np.fft.fft(sig) / t.shape[0] 

        freq = np.fft.fftfreq(t.shape[0], d=dt)

        firstNegInd = np.argmax(freq < 0)
        freqAxisPos = freq[:firstNegInd]
        sigFFTPos = 2 * sigFFT[:firstNegInd] 

        if plot:
            plt.figure()
            plt.plot(freqAxisPos, 20*np.log10(np.abs(sigFFTPos)/2**14))
            plt.ylim([-140, -80])
            plt.xscale("log")
            plt.xlabel(xLabel)
            plt.ylabel("U.A.")
            plt.title("Analytic FFT plot")
            plt.show()

        self.x = freqAxisPos /10e5
        self.y = 20*np.log10(np.abs(sigFFTPos)/2**14)
#wf=readwave(plot=True).wf
#f=fft   (wf,dt=None,plot=True)     
        
class mean_fft:
    def __init__(self, data,label=None,plot=False):
        np.seterr(divide = 'ignore') 
        fft_list_x=[]
        fft_list_y=[]
        std_list=[]
        for k in range(len(data)):
            fft_list_x.append(fft(data[k]).x)
            fft_list_y.append(fft(data[k]).y)
            std_list.append(np.std(data[k],axis=0))
        self.x=np.mean(fft_list_x,axis=0) 
        self.y=np.mean(fft_list_y,axis=0)
        self.stdx=np.mean(std_list,axis=0)
        if plot:
            plt.rcParams["figure.figsize"] = (10,7)
            plt.figure()
            plt.plot(self.x,self.y,linewidth=1,label=f'{label} \t rms={np.round(self.stdx,2)}'.expandtabs())
            plt.ylim([min(self.y)-10, -80])
            #plt.xscale("log")
            # plt.xlim([0.2, 3])
            plt.legend()
            plt.ylabel("dBFS")
            plt.xlabel("MHz")
            plt.title("FFT")
            plt.tight_layout()

def calculate_rms_deviation(array):
    mean = np.mean(array)
    deviations = array - mean
    squared_deviations = deviations ** 2
    mean_squared_deviation = np.mean(squared_deviations)
    rms_deviation = np.sqrt(mean_squared_deviation)
    return rms_deviation

from init import write_and_read

def tune_offset(thing, i, afe, tolerance=50, target_pedestal=8192):
    #thing = OEI(f"10.73.137.{i}")
    print("DAPHNE firmware version %0X" % thing.read(0x9000,1)[2])
    print(f'DAPHNE ip address = 10.73.137.{i}')
    channels=[4,5,6,7]
    for ch in channels:
        thing.write(0x2000, [1234]) # software trigger, all spy buffers capture
        a=np.average(thing.read(0x40000000+(afe*0x100000)+(ch*0x10000),50)[40:])
        #print (a)
        offset=1118
        pedestal=target_pedestal
        while tolerance<=abs(pedestal-a):
            offset=offset+((pedestal-a)/abs(pedestal-a))
            write_and_read(thing, f'WR OFFSET CH {afe*8+ch} V {offset}',get_response=True) 
            thing.write(0x2000, [1234]) # software trigger, all spy buffers capture
            a=np.average(thing.read(0x40000000+(afe*0x100000)+(ch*0x10000),50)[40:])

            print (a) 
        print("AFE%d[%d]: " % (afe,ch),end="")
        for x in thing.read(0x40000000+(afe*0x100000)+(ch*0x10000),15)[3:]:
            print("%04X " % x,end="")
        print()
    print()
           
    thing.close()

In [ ]:
### take data at multiple VGAIN values and calculate RMS and FFT

# make a list of VGAIN settings to try
VGAIN_SETTINGS = [0]
i = 100 # ip endpoint

channel = 0
AFE = 0
nWvfms = 1000

FFT_data = []
RMS_data = []
ADC_data = []

turn_on_OIs = False
set_VGAIN = True
tune_offset = False

# try each VGAIN value and take data with each setting
for VGAIN in VGAIN_SETTINGS:
    thing = OEI(f"10.73.137.{i}")
    
    if turn_on_OIs:
        # turn on offset integrators, should tune offset also to close to 8196
        write_and_read(thing, 'WR AFE '+ str(int(AFE)) + ' REG 52 V 16896', get_response=True)
        write_and_read(thing, 'WR AFE '+ str(int(AFE)) + ' REG 51 V 0', get_response=True)

    if set_VGAIN:
        write_and_read(thing, 'WR AFE '+ str(int(AFE)) + f' VGAIN V {VGAIN}', get_response=True)
        
    if tune_offset:
        tune_offset(thing, i, AFE, target_pedestal=8196, tolerance=100)
    
    progress=trange(nWvfms, unit='waveforms',desc = f'Reading from endpoint {i}:')
    d = [readwave(ep=i, ch=channel,afe=AFE).wf for _ in progress ]
    RMS_data.append(calculate_rms_deviation(np.array(d).flatten()))
    FFT_data.append(mean_fft(d))
    ADC_data.append(np.array(d).flatten())
    
for RMS in RMS_data:
    print(f'RMS value: {RMS:.3f}')

In [ ]:
# optionally save FFT, RMS, and ADC data to npz file for later analysis
BIAS=47
np.savez(f'FFT_RMS_ADC_data_ch{channel}_afe{AFE}_March12024_VGAIN{VGAIN_SETTINGS[0]}_BOTHOffsetIntegratorsOFF.npz', FFT_data=np.array(FFT_data), RMS_data=np.array(RMS_data), ADC_data=np.array(ADC_data))

In [ ]:
### example of loading npz data

#np.load('FFT_RMS_ADC_data_ch0_afe0_variousVGAIN_offsetIntegratorsOn.npz',allow_pickle=True)['ADC_data']

In [ ]:
VGAIN_SETTINGS = [600]
i = 100
AFEs = [0,1,2,3,4]

VGAIN=VGAIN_SETTINGS[0]
# to set VGAIN on each AFE
thing = OEI(f"10.73.137.{i}")
for AFE in AFEs:
    write_and_read(thing, 'WR AFE '+ str(int(AFE)) + f' VGAIN V {VGAIN}', get_response=True)
    

In [ ]:
thing = OEI(f"10.73.137.{i}")
AFEs=[0]
for AFE in AFEs:
    # first turn on offset integrators
    #write_and_read(thing, 'WR AFE '+ str(int(AFE)) + ' REG 52 V 16896', get_response=True)
    #write_and_read(thing, 'WR AFE '+ str(int(AFE)) + ' REG 51 V 0', get_response=True)
    #write_and_read(thing, 'WR AFE '+ str(int(AFE)) + ' REG 52 V 21056', get_response=True)
    #write_and_read(thing, 'WR AFE '+ str(int(AFE)) + ' REG 51 V 16', get_response=True)
    write_and_read(thing, 'WR AFE '+ str(int(AFE)) + ' REG 52 V 20992', get_response=True)
    write_and_read(thing, 'WR AFE '+ str(int(AFE)) + ' REG 51 V 16', get_response=True)
    


In [ ]:
### plot FFTs

np.seterr(divide = 'ignore') 
plt.rcParams["figure.figsize"] = (10,7)
plt.figure()

for j in range(len(VGAIN_SETTINGS)):
    plt.plot(FFT_data[j].x,FFT_data[j].y,linewidth=1,label=f'VGAIN = {VGAIN_SETTINGS[j]}')
plt.legend()

plt.ylim([-115, -80])
plt.ylabel("dBFS")
plt.xlabel("MHz")
#plt.xlim(0, 3.5)
plt.title(f'CSU DAPHNE FFT, channel {channel}, AFE {AFE}')
#plt.title(f"Noise tests on DAPHNE V2A ch {Channel+AFE[]*8}")
plt.grid(axis='x',color="0.95")
plt.tight_layout()
#plt.savefig(f'FFTs_ch{channel}_afe{AFE}_variousVGAIN_offsetIntegratorsOn.pdf')
plt.legend()

In [ ]:
### plot RMS data points (particularly useful for multiple VGAIN values)

np.seterr(divide = 'ignore') 
plt.rcParams["figure.figsize"] = (10,7)
plt.figure()

plt.scatter(VGAIN_SETTINGS, RMS_data, linewidth=1)
plt.legend()
#plt.ylim([-115, -87])
plt.ylabel("ADC RMS")
plt.xlabel("VGAIN")
plt.title(f'CSU DAPHNE Waveform RMS, channel {channel}, AFE {AFE} (offset integrators OFF)')
#plt.title(f"Noise tests on DAPHNE V2A ch {Channel+AFE[]*8}")
#plt.grid(axis='x',color="0.95")
plt.tight_layout()
#plt.savefig(f'RMS_plot_ch{channel}_afe{AFE}_variousVGAIN_offsetIntegratorsOn.pdf')
plt.legend()

In [ ]:
### plot histograms of ADC data for each VGAIN value

import matplotlib.pyplot as plt
import numpy as np

np.seterr(divide='ignore') 
plt.rcParams["figure.figsize"] = (10, 6)

# Determine the number of rows and columns for subplots
num_plots = len(VGAIN_SETTINGS)
num_cols = 4  # Adjust the number of columns as needed
num_rows = (num_plots + num_cols - 1) // num_cols

fig, axes = plt.subplots(num_rows, num_cols, figsize=(14, 3*num_rows))

# Flatten axes if it's a single row or column
if num_rows == 1:
    axes = axes.reshape(1, -1)
if num_cols == 1:
    axes = axes.reshape(-1, 1)

for j, ax in enumerate(axes.flat):
    if j < num_plots:
        ax.hist(ADC_data[j].astype('int'), range=(np.min(ADC_data[j]), np.max(ADC_data[j])), bins=np.max(ADC_data[j])-np.min(ADC_data[j]), label=f'VGAIN = {VGAIN_SETTINGS[j]}', alpha=0.5)
        ax.legend(fontsize='small')
        if j == 0:
            nticks=10
        elif j == 1:
            nticks=10
        elif j==2:
            nticks=10
        else:
            nticks=5
        ax.set_xticks(np.arange(np.min(ADC_data[j]), np.max(ADC_data[j]), nticks))
        ax.set_ylabel("Counts")
        ax.set_xlabel("ADCs")
        
#ax.set_title(f'CSU DAPHNE ADC Histogram, channel {channel}, AFE {AFE} (offset integrators ON)')
ax.set_rasterization_zorder(2)
# Hide any unused subplots
for j in range(num_plots, num_rows * num_cols):
    axes.flat[j].axis('off')
fig.suptitle(f'CSU DAPHNE ADC Histograms, channel {channel}, AFE {AFE} (offset integrators ON)')
plt.tight_layout()
#plt.savefig(f'ADC_histograms_ch{channel}_afe{AFE}_variousVGAIN_offsetIntegratorsOn.pdf')

plt.show()


In [ ]:
### define function for converting h5 to csv

import h5py
import numpy as np
start_index = 1000
stop_index = 2000
#filename = "daphne_fnal_data/config4/daphne_fnal_LED_config4_VGAIN900"
def convert_h5py_to_csv(h5py_filename, csv_filename):
    with h5py.File(h5py_filename, 'r') as f:
        # Load all waveforms into memory
        waveforms_data = np.array(f['waveforms']['adc'][:], dtype='i4')
        waveforms_data = waveforms_data[:, start_index:stop_index]
    # Reshape the waveforms_data array to have each waveform in a column
    reshaped_data = waveforms_data

    # Write the reshaped data into a CSV file
    output_filename = csv_filename
    np.savetxt(output_filename, reshaped_data, delimiter=',', fmt='%d')

In [ ]:
### convert multiple files to csv from h5

import os
import fnmatch

def get_files_in_directory(directory, pattern):
    file_list = []
    for root, dirs, files in os.walk(directory):
        for filename in files:
            if fnmatch.fnmatch(filename, pattern):
                file_list.append(os.path.join(root, filename).replace('\\', '/'))
    return file_list

# Example usage:
directory = 'daphne_fnal_data'
pattern = 'daphne_fnal_config5*.h5'  # Example pattern to match all files ending with '.txt'

matching_files = get_files_in_directory(directory, pattern)
for file in matching_files:
    print(file)
    convert_h5py_to_csv(file, f"{file.split('.h5')[0]}.csv")


In [ ]:
channel=0
AFE=0
VGAIN=900
data_OI_BOTH_OFF = np.load(f'FFT_RMS_ADC_data_ch{channel}_afe{AFE}_March12024_VGAIN0_BOTHOffsetIntegratorsOFF.npz',allow_pickle=True)
data_OI_BOTH_ON = np.load(f'FFT_RMS_ADC_data_ch{channel}_afe{AFE}_March12024_VGAIN0_BothOffsetIntegratorsON.npz',allow_pickle=True)
data_OI_PGA_ON = np.load(f'FFT_RMS_ADC_data_ch{channel}_afe{AFE}_March12024_VGAIN0_OnlyPGAOffsetIntegratorsON.npz',allow_pickle=True)
data_OI_LNA_ON = np.load(f'FFT_RMS_ADC_data_ch{channel}_afe{AFE}_March12024_VGAIN0_OnlyLNAOffsetIntegratorsON.npz',allow_pickle=True)


In [ ]:
%matplotlib notebook
plt.rcParams["figure.figsize"] = (8,6)
plt.figure()

plt.plot(data_OI_BOTH_OFF['FFT_data'][0].x,data_OI_BOTH_OFF['FFT_data'][0].y,linewidth=1,label='Both Offset Integrators OFF')
plt.plot(data_OI_BOTH_ON['FFT_data'][0].x,data_OI_BOTH_ON['FFT_data'][0].y,linewidth=1,label='Both Offset Integrators ON')
plt.plot(data_OI_PGA_ON['FFT_data'][0].x,data_OI_PGA_ON['FFT_data'][0].y,linewidth=1,label='Only PGA Offset Integrator ON')
plt.plot(data_OI_LNA_ON['FFT_data'][0].x,data_OI_LNA_ON['FFT_data'][0].y,linewidth=1,label='Only LNA Offset Integrator ON')
plt.legend()

plt.ylim([-115, -80])
plt.ylabel("dBFS")
plt.xlabel("MHz")
#plt.xlim(0, 3.5)
plt.title(f'CSU DAPHNE FFT, channel {channel}, AFE {AFE}, March 1 2024 Offset Integrator Test')
#plt.title(f"Noise tests on DAPHNE V2A ch {Channel+AFE[]*8}")
plt.grid(axis='x',color="0.95")
plt.tight_layout()
#plt.savefig(f'FFTs_ch{channel}_afe{AFE}_variousVGAIN_offsetIntegratorsOn.pdf')
plt.legend()

In [ ]:
%matplotlib notebook
plt.rcParams["figure.figsize"] = (12,2)

plot_one = False

if plot_one:
    plt.figure()
    ADC_data_OI_BOTH_ON = (data_OI_BOTH_ON['ADC_data'][0] - int(np.mean(data_OI_BOTH_ON['ADC_data'][0]))).astype('int')
    plt.hist(ADC_data_OI_BOTH_ON.astype('int'), range=(np.min(ADC_data_OI_BOTH_ON), np.max(ADC_data_OI_BOTH_ON)), bins=np.max(ADC_data_OI_BOTH_ON)-np.min(ADC_data_OI_BOTH_ON), label=f'Both Offset Integrators ON', alpha=0.5)
    ADC_data_OI_BOTH_OFF = (data_OI_BOTH_OFF['ADC_data'][0] - int(np.mean(data_OI_BOTH_OFF['ADC_data'][0]))).astype('int')
    plt.hist(ADC_data_OI_BOTH_OFF.astype('int'), range=(np.min(ADC_data_OI_BOTH_OFF), np.max(ADC_data_OI_BOTH_OFF)), bins=np.max(ADC_data_OI_BOTH_OFF)-np.min(ADC_data_OI_BOTH_OFF), label=f'Both Offset Integrators OFF', alpha=0.5)
    ADC_data_OI_PGA_ON = (data_OI_PGA_ON['ADC_data'][0] - int(np.mean(data_OI_PGA_ON['ADC_data'][0]))).astype('int')
    plt.hist(ADC_data_OI_PGA_ON.astype('int'), range=(np.min(ADC_data_OI_PGA_ON), np.max(ADC_data_OI_PGA_ON)), bins=np.max(ADC_data_OI_PGA_ON)-np.min(ADC_data_OI_PGA_ON), label=f'Only PGA Offset Integrator', alpha=0.5)
    ADC_data_OI_LNA_ON = (data_OI_LNA_ON['ADC_data'][0] - int(np.mean(data_OI_LNA_ON['ADC_data'][0]))).astype('int')
    plt.hist(ADC_data_OI_LNA_ON.astype('int'), range=(np.min(ADC_data_OI_LNA_ON), np.max(ADC_data_OI_LNA_ON)), bins=np.max(ADC_data_OI_LNA_ON)-np.min(ADC_data_OI_LNA_ON), label=f'Only LNA Offset Integrator ON', alpha=0.5)
    
    plt.legend()
    plt.xlabel('ADC - mean ADC')
    plt.ylabel('counts')
    plt.title(f'VGAIN = {VGAIN}')
else:
    fig, axes = plt.subplots(1, 4, figsize=(15, 4))
    ADC_data_OI_BOTH_ON = (data_OI_BOTH_ON['ADC_data'][0]).astype('int')
    axes[0].hist(ADC_data_OI_BOTH_ON.astype('int'), range=(np.min(ADC_data_OI_BOTH_ON), np.max(ADC_data_OI_BOTH_ON)), bins=np.max(ADC_data_OI_BOTH_ON)-np.min(ADC_data_OI_BOTH_ON), label=f'Both Offset Integrators ON', alpha=0.5)
    ADC_data_OI_BOTH_OFF = (data_OI_BOTH_OFF['ADC_data'][0]).astype('int')
    axes[1].hist(ADC_data_OI_BOTH_OFF.astype('int'), range=(np.min(ADC_data_OI_BOTH_OFF), np.max(ADC_data_OI_BOTH_OFF)), bins=np.max(ADC_data_OI_BOTH_OFF)-np.min(ADC_data_OI_BOTH_OFF), label=f'Both Offset Integrators OFF', alpha=0.5)
    ADC_data_OI_PGA_ON = (data_OI_PGA_ON['ADC_data'][0]).astype('int')
    axes[2].hist(ADC_data_OI_PGA_ON.astype('int'), range=(np.min(ADC_data_OI_PGA_ON), np.max(ADC_data_OI_PGA_ON)), bins=np.max(ADC_data_OI_PGA_ON)-np.min(ADC_data_OI_PGA_ON), label=f'Only PGA Offset Integrator', alpha=0.5)
    ADC_data_OI_LNA_ON = (data_OI_LNA_ON['ADC_data'][0]).astype('int')
    axes[3].hist(ADC_data_OI_LNA_ON.astype('int'), range=(np.min(ADC_data_OI_LNA_ON), np.max(ADC_data_OI_LNA_ON)), bins=np.max(ADC_data_OI_LNA_ON)-np.min(ADC_data_OI_LNA_ON), label=f'Only LNA Offset Integrator ON', alpha=0.5)
    
    axes[0].set_xlabel('ADC')
    axes[1].set_xlabel('ADC')
    axes[0].set_ylabel('counts')
    #axes[1].set_ylabel('counts')
    axes[0].set_title(f'Both Offset Integrators ON', fontsize=9)
    axes[1].set_title(f'Both Offset Integrators OFF', fontsize=9)
    axes[2].set_title('Only PGA Offset Integrator ON', fontsize=9)
    axes[3].set_title('Only LNA Offset Integrator ON', fontsize=9)
    #plt.savefig('daphne_offset_integrator_test.pdf')

In [ ]:
print(f"RMS offset integrators off = {data_OI_OFF['RMS_data'][0]:.3f}")
print(f"RMS offset integrators on = {data_OI_ON['RMS_data'][0]:.3f}")